구글 코랩에서 실행하는 경우, cv2.imshow("VideoFrame", frame) 구문을 다음과 같이 수정합니다.

```python
from google.colab.patches import cv2_imshow

# ...

cv2_imshow(frame) # cv2.imshow("VideoFrame", frame)을 대체합니다.
```

In [1]:
from ultralytics import YOLO


model = YOLO("./models/yolov8m-pose.pt")

/Users/chohi/project/ai/DeepLearningDeepen/DeepLearningAdv/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import cv2


capture = cv2.VideoCapture("./datasets/woman.mp4")
while cv2.waitKey(10) < 0:
    if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
        capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

    ret, frame = capture.read()
    cv2.imshow("VideoFrame", frame)

capture.release()
cv2.destroyAllWindows()

In [9]:
import torch


def predict(frame, iou=0.7, conf=0.25):
    results = model(
        source=frame,
        device="0" if torch.cuda.is_available() else "cpu",
        iou=0.7,
        conf=0.25,
        verbose=False,
    )
    result = results[0]
    return result

In [10]:
def draw_boxes(result, frame):
    for boxes in result.boxes:
        x1, y1, x2, y2, score, classes = boxes.data.squeeze().cpu().numpy()
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)
    return frame

In [11]:
import cv2


capture = cv2.VideoCapture("./datasets/woman.mp4")
while cv2.waitKey(10) < 0:
    if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
        capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

    ret, frame = capture.read()
    result = predict(frame)
    frame = draw_boxes(result, frame)

    cv2.imshow("VideoFrame", frame)

capture.release()
cv2.destroyAllWindows()

/Users/chohi/project/ai/DeepLearningDeepen/DeepLearningAdv/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import cv2
import numpy as np

def draw_keypoints(result, frame):
    """
    YOLOv8 Pose 결과를 OpenCV로 시각화하는 함수
    :param result: YOLOv8 추론 결과 (results[0])
    :param frame: 원본 이미지 (numpy array)
    :return: keypoint가 그려진 이미지
    """
    if not hasattr(result, 'keypoints') or result.keypoints is None:
        return frame

    kps_tensor = result.keypoints.data  # shape: (num_persons, num_kpts, 3)
    kps = kps_tensor.cpu().numpy()

    for person in kps:
        for idx, (x, y, conf) in enumerate(person):
            if conf > 0.5:
                cv2.circle(frame, (int(x), int(y)), 3, (0, 0, 255), cv2.FILLED)
                cv2.putText(frame, str(idx), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

    return frame


In [15]:
import cv2


capture = cv2.VideoCapture("./datasets/woman.mp4")
while cv2.waitKey(10) < 0:
    if capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT):
        capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

    ret, frame = capture.read()
    result = predict(frame)
    frame = draw_boxes(result, frame)
    frame = draw_keypoints(result, frame)
    cv2.imshow("VideoFrame", frame)

capture.release()
cv2.destroyAllWindows()